<h1 align="center"> Week 4 - Transformer </h1>

<p style="text-align:center">汇报人：xxx</p>

---


###  建议学习资料：

- [李沐动手学深度学习61-68](https://space.bilibili.com/1567748478/lists/358497?type=series)
- [原文Attention Is All You Need](https://arxiv.org/abs/1706.03762)

###  本周任务清单：

1.  **理论学习**
   - 学习 Transformer 模型的基本原理  

2.  **实践任务**
   - 构建一个 CNN+Transformer 模型，在 RML2016.10a 数据集上进行信号分类
   - 构建一个 纯Transformer 模型，在 RML2016.10a 数据集上进行信号分类
   - 比较两种模型训练的差异

3.  **提交要求**
   - 提交一个 `.ipynb` 文件，文件内容需包括：
     - 每个理论学习点下方填写的你整理的学习内容（可结合简单代码辅助理解）
     - 所有实践任务的结果展示（简单的实践任务代码可直接写在该文档）
   - 注：对文字排版无具体要求；使用 Jupyter Notebook 打开，图片可直接通过粘贴插入 Markdown  
   - 文件命名格式：`Week4_姓名.ipynb`

---

### 理论学习

#### 1. 位置编码（Positional Encoding）
- 理解位置编码的作用：在 Transformer 中，如何通过位置编码为模型引入序列的位置信息
- 位置编码的两种常见方式：正弦函数（Sinusoidal Encoding）与可学习位置编码（Learned Positional Encoding）

#### 2. (多头)自注意力机制（Attention Mechanism）
- 理解自注意力机制（Self-Attention）如何通过计算查询（Query）、键（Key）和值（Value）来生成加权表示
- 学习点积注意力（Scaled Dot-Product Attention）的原理和公式

#### 3. 交叉注意力机制（Cross-Attention Mechanism）
- 理解交叉注意力机制的概念，特别是在Transformer 解码器中，如何通过交叉注意力机制对编码器的输出进行加权
- 对比交叉注意力与自注意力的差异，理解它们各自的应用场景

#### 4. Transformer 编码器与解码器
- 了解 Transformer 中编码器（Encoder）与解码器（Decoder）的工作原理和结构

#### 5. 残差连接与层归一化（Residual Connection & Layer Normalization）
- 理解 Transformer 中的残差连接如何帮助缓解梯度消失问题，提升训练稳定性
- 学习层归一化的原理及其在 Transformer 中的应用，确保网络的训练能够更加平稳
- 对比Batch Normalization 和 Layer Normalization

---

### 实践任务

#### 任务一：构建一个 CNN+Transformer 模型（用于调制识别）

**目标：** 使用 PyTorch 构建CNN+Transformer 模型，对调制信号数据进行分类识别任务。

**要求：**

- **自行学习Transforemr代码**
- 使用提供的数据加载代码（输入数据维度为 `[batch_size, 2, 128]`，即2个通道、128个时间步）
- 模型部分自行设计：
  - CNN：参考ResNet自行设计(CNN提取到的特征怎么转化为适合Transformer的输入自行调整)
  - Transformer：d_model=128, num_heads=8, hidden_dim=256, num_layers=2（仅供参考）
  - 全连接层：输入128维，输出11类（使用平均操作得到全局特征 或者 class token代表全局特征进行分类）
- 损失函数：`nn.CrossEntropyLoss()`
- 优化器、学习率、EarlyStop：自行设计
- 可视化训练过程中的 `loss` 和 `accuracy` 曲线
- 输出模型在训练集与测试集上的分类准确率

In [ ]:
import torch.utils.data as Data
import pandas as pd
import torch
import numpy as np

def to_num(y):
    label = []
    classes = ['8PSK','AM-DSB','AM-SSB','BPSK','CPFSK','GFSK','PAM4','QAM16','QAM64','QPSK', 'WBFM']
    for j in range(len(y)):
        label.append(classes.index(y[j]))
    label = torch.tensor(label)
    return label

def RML_dataloader(train_rate, valid_rate, test_rate, batch_size=64, random_seed=2024, num_workers=2):

    data = pd.read_pickle('RML2016.10a_dict.pkl')
    snrs, mods = map(lambda j: sorted(list(set(map(lambda x: x[j], data.keys())))), [1, 0])
    X = []
    label = []
    for mod in mods:
        for snr in snrs:
            X.append(data[(mod, snr)])
            for i in range(data[(mod, snr)].shape[0]):  label.append((mod, snr))
    X = np.vstack(X)

    n_examples = X.shape[0]
    n_train = int(train_rate * n_examples)

    train_idx = []
    test_idx = []
    val_idx = []

    np.random.seed(random_seed)
    Slices_list = np.linspace(0, n_examples, num=len(mods) * len(snrs) + 1)

    for k in range(0, Slices_list.shape[0] - 1):
        train_idx_subset = np.random.choice(
            range(int(Slices_list[k]), int(Slices_list[k + 1])), size=int(n_train / (len(mods) * len(snrs))),
            replace=False)
        Test_Val_idx_subset = list(set(range(int(Slices_list[k]), int(Slices_list[k + 1]))) - set(train_idx_subset))
        test_idx_subset = np.random.choice(Test_Val_idx_subset,
                                           size=int(
                                               (n_examples - n_train) * test_rate / (
                                                       (len(mods) * len(snrs)) * (test_rate + valid_rate))),
                                           replace=False)
        val_idx_subset = list(set(Test_Val_idx_subset) - set(test_idx_subset))

        train_idx = np.hstack([train_idx, train_idx_subset])
        val_idx = np.hstack([val_idx, val_idx_subset])
        test_idx = np.hstack([test_idx, test_idx_subset])

    train_indices = train_idx.astype('int64')
    val_indices = val_idx.astype('int64')
    test_indices = test_idx.astype('int64')

    X_train = torch.tensor(X[train_indices])  # [N, 2, 128]
    X_val = torch.tensor(X[val_indices])
    X_test = torch.tensor(X[test_indices])
    y_train, y_val, y_test = [], [], []
    snr_train, snr_val, snr_test = [], [], []

    for i in train_indices:
        y_train.append(label[i][0])
        snr_train.append(label[i][1])
    for i in val_indices:
        y_val.append(label[i][0])
        snr_val.append(label[i][1])
    for i in test_indices:
        y_test.append(label[i][0])
        snr_test.append(label[i][1])

    y_train = to_num(y_train)
    y_val = to_num(y_val)
    y_test = to_num(y_test)

    snr_test = torch.tensor(snr_test)
    snr_val = torch.tensor(snr_val)
    snr_train = torch.tensor(snr_train)

    torch_dataset = Data.TensorDataset(X_train, y_train, snr_train)
    loader1 = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    torch_dataset = Data.TensorDataset(X_val, y_val, snr_val)
    loader2 = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=batch_size
        shuffle=True,
        num_workers=num_workers,
    )

    torch_dataset = Data.TensorDataset(X_test, y_test, snr_test)
    loader3 = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    return loader1, loader2, loader3  # train,valid,test

if __name__ == '__main__':
    train_loader, valid_loader, test_loader = RML_dataloader(0.6, 0.2, 0.2, batch_size=500, random_seed=2024)

#### 任务二：构建一个 纯Transformer 模型（用于调制识别）

**目标：** 使用 PyTorch 构建 Transformer 模型，对调制信号数据进行分类识别任务。

**要求：**

- 使用提供的数据加载代码（输入数据维度为 `[batch_size, 2, 128]`，即2个通道、128个时间步）
- 模型部分自行设计：
  - 嵌入层（Embedding）：使用 `nn.Conv1d(2, 64, kernel_size=16, stride=8)` 
  - Transformer：d_model=128, num_heads=8, hidden_dim=256, num_layers=2（仅供参考）
  - 全连接层：输入128维，输出11类（使用平均操作得到全局特征 或者 class token代表全局特征进行分类）
- 损失函数：`nn.CrossEntropyLoss()`
- 优化器、学习率、EarlyStop：自行设计
- 可视化训练过程中的 `loss` 和 `accuracy` 曲线
- 输出模型在训练集与测试集上的分类准确率

#### 任务三：结果比较

**目标：** 比较上述两种架构在训练过程中、训练结果上有何不同。

**要求：**

- 多方面比较二者区别